In [ ]:
!pip install -U kornia
!pip install kornia-moons
!pip install omegaconf

In [ ]:
!pip install gdown
!mkdir weights

In [ ]:
#OpenGlue
!git clone https://github.com/ucuapps/OpenGlue
!gdown https://drive.google.com/drive/folders/1N-x_-KzSFgCVO58YeCA3B6AHC2n8JK9J -O weights --folder

In [ ]:
#SuperPoint
!git clone https://github.com/magicleap/SuperPointPretrainedNetwork

In [ ]:
import sys
import os

package_path = '/kaggle/working/'
package_list = ['OpenGlue']

for package in package_list:
    sys.path.append( os.path.join( package_path, package ) )

In [ ]:
import cv2
import os
import numpy as np
import pandas as pd
from omegaconf import OmegaConf

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [ ]:
import torch

from models.features import get_feature_extractor
from models.superglue.superglue import SuperGlue
from inference import (load_torch_image, OpenGlueMatcher)

In [ ]:
 def draw_matches(img_path0, kp0, img_path1, kp1):
    kp0 = [cv2.KeyPoint(kp0[i, 0], kp0[i, 1], 1) for i in range(len(kp0))]
    kp1 = [cv2.KeyPoint(kp1[i, 0], kp1[i, 1], 1) for i in range(len(kp1))]
    matches = [cv2.DMatch(i, i, 1) for i in range(len(kp0))]

    img0 = cv2.imread(img_path0) 
    img1 = cv2.imread(img_path1) 
    
    img0 = cv2.cvtColor(img0, cv2.COLOR_BGR2RGB)
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)

    match_img = cv2.drawMatches(img0, kp0, img1, kp1, matches, None, flags=2)

    plt.figure(figsize=(24, 10))
    plt.imshow(match_img)
    plt.xticks([])
    plt.yticks([])
    plt.show()

In [ ]:
def build_matcher(config_path,
                    features_config_path,
                    checkpoint_path,
                    features_checkpoint_path,
                    device=torch.device('cpu'),
                    max_features=2048,
                    resize_to='original'):

    config = OmegaConf.load(config_path)
    features_config = OmegaConf.load(features_config_path)

    config['features'] = features_config
    config['features']['max_keypoints'] = max_features
    config['features']['parameters']['weights'] = features_checkpoint_path

    if isinstance(resize_to, str):
        assert resize_to in ["as in config", "original"]
        if resize_to == 'original':
            config['data']['target_size'] = None
    else:
        assert len(resize_to) == 2
        assert resize_to[0] > 0
        assert resize_to[1] > 0
        config['data']['target_size'] = resize_to

    if 'weights' in config['superglue'].keys():
        del config['superglue']['weights']

    # Initialize models & load weights
    local_features_extractor = get_feature_extractor(config['features']['name'])(**config['features']['parameters'])
    local_features_extractor.to(device)

    state_dict = torch.load(str(checkpoint_path), map_location='cpu')['state_dict']
    for key in list(state_dict.keys()):
        state_dict[key.replace('superglue.', '')] = state_dict.pop(key)

    superglue = SuperGlue(config['superglue'])
    message = superglue.load_state_dict(state_dict)
    print(message)
    superglue.to(device)

    matcher = OpenGlueMatcher(local_features_extractor, superglue, config)
    matcher.to(device)

    return matcher

In [ ]:
def FlattenMatrix(M, num_digits = 8):
    '''Convenience function to write CSV files.'''    
    return ' '.join([f'{v:.{num_digits}e}' for v in M.flatten()])

def get_fundamental_matrix(kpts0, kpts1):  
    if len(kpts0) > 7:
        F, inliers = cv2.findFundamentalMat(kpts0, kpts1, cv2.USAC_MAGSAC, 0.2, 0.99999, 100000)
    else:
        F = np.zeros((3, 3))  
    return F

In [ ]:
dataset_path = '/kaggle/input/image-matching-challenge-2022/'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu', 0)

In [ ]:
config_path = os.path.join( package_path, 'OpenGlue/config/config_cached_sp_magicleap.yaml' )
checkpoint_path = os.path.join( package_path, 'weights/SuperPoint/openglue_superpoint-kitti.ckpt' )

features_config_path = os.path.join( package_path, 'OpenGlue/config/features/superpoint_magicleap.yaml' )
features_checkpoint_path = os.path.join( package_path, 'SuperPointPretrainedNetwork/superpoint_v1.pth' )

In [ ]:
matcher = build_matcher(
            config_path,
            features_config_path,
            checkpoint_path,
            features_checkpoint_path,
            device=device,
            max_features=2048)

In [ ]:
test_df = pd.read_csv(f'{dataset_path}/test.csv')
test_df

In [ ]:
F_list = []

for i, row in tqdm( test_df.iterrows(), total=len(test_df) ):
    sample_id  = row.sample_id
    batch_id   = row.batch_id
    image_1_id = row.image_1_id
    image_2_id = row.image_2_id
    
    img_path0 = f'{dataset_path}/test_images/{batch_id}/{image_1_id}.png'
    img_path1 = f'{dataset_path}/test_images/{batch_id}/{image_2_id}.png'
    
    timg0 = load_torch_image( img_path0, device=device )
    timg1 = load_torch_image( img_path1, device=device )

    with torch.no_grad():
        out = matcher({"image0": timg0, "image1": timg1})

    kp0 = out['keypoints0'].detach().cpu().numpy()
    kp1 = out['keypoints1'].detach().cpu().numpy()
    
    F = get_fundamental_matrix(kp0, kp1)
    F_list.append( FlattenMatrix(F) )
    
    if i < 1:
        draw_matches(img_path0, kp0, img_path1, kp1)

In [ ]:
submission_df = pd.DataFrame()

submission_df['sample_id'] = test_df['sample_id']
submission_df['fundamental_matrix'] = F_list

submission_df.to_csv('submission.csv',index=False)
submission_df